In [ ]:
import socket
import time
import pickle #Pickle is a module used in python to serialize and deserialize the objects. 
#In socket programming it is used for communicating between the programs both locally and remotely

serverip = '192.168.159.1'
serverport = 50000
#In the above two lines we initiate the ip address and port number of the server

client = None
connectionsocket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM) #socket.socket() -> creates a socket object.
# Parameters -> Address family and socket type, AF_INET -> Internet address family for IPv4, SOCK_DGRAM -> UDP data type 
myaddress = (serverip, serverport)
# Binding of socket
connectionsocket.bind(myaddress)


def Recv_con():
    while True:
            connectionsocket.settimeout(None)
            print("Waiting for connection")
            data, client = connectionsocket.recvfrom(2048) # Client -> Gives client ipaddress and port number, data -> Inform of bytes
            tcpHeader = pickle.loads(data) # loads method of pickle module de-serialize the data stream 
            Make_con(tcpHeader, client)

def Make_con(header, client):
    print("Validating connection ...")
    if header.syn == True:
        print('Connection received, sending confirmation ...')
        time.sleep(1)
        connectionobject = tcp_header(True, True)
        data_byte = pickle.dumps(connectionobject) # dumps method in pickle module serialize the data stream
        connectionsocket.sendto(data_byte, client) # sendto method sends the UDP packet the client socket
        Recv_files()


def Send_conf(tcpheader, client):
    print("Sending receipt confirmation ...")
    time.sleep(2)
    confirmationobject = tcpheader
    confirmationobject.file = None
    confirmationobject.acknowledgement_number = confirmationobject.sequence_number + 1
    data_byte = pickle.dumps(confirmationobject) 
    connectionsocket.sendto(data_byte, client)


def Recv_files():
    global client
    try:
        connectionsocket.settimeout(5)
        tcpheader = tcp_header(False, False)
        while tcpheader.fin is False: # fin flag indicates the end of data transmission
            print("Receiving File ...")
            data, client_ = connectionsocket.recvfrom(664000) # receives the data the socket named and stores it in the buffer
            if client is None:
                client = client_
            tcpheader = pickle.loads(data)
            buffer = open("copy.txt", "ab") # Opens a text file with the name copy and appends the data in binary mode
            if tcpheader.file is not None and client == client_:
                buffer.write(tcpheader.file) # writes the text into specified file 
                Send_conf(tcpheader, client)
        buffer.close()
        client = None
        print("File received successfully!")
    except socket.timeout:
        print("TIMEOUT, ending connection!")
        client = None
    except:
        print("Corrupted file, terminating connection!")
        client = None

# Defining the TCP Header class
# Sequence Number is used to identify each of byte of data
# Acknowledgement Number provides the acknowledgement
class tcp_header:
    sequence_number = None
    acknowledgement_number = None
    syn = False
    ack = False
    fin = False
    rst = False
    file = None

    def __init__(self, syn, ack):
        self.syn = syn
        self.ack = ack
flag=0
if(flag==0):
    Recv_con()
    flag+=1
else:
    exit()

Waiting for connection
Validating connection ...
Connection received, sending confirmation ...
Receiving File ...
Sending receipt confirmation ...
Receiving File ...
File received successfully!
Waiting for connection
